In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import re
import string

In [2]:
df = pd.read_csv("train.csv")

In [3]:
df.drop_duplicates(subset = ['text','target'], keep = 'first', inplace = True, ignore_index = True)

In [4]:
duplicates_record = df[df.duplicated(['text'], keep=False)]
duplicates_record

,id,keyword,location,text,target
606,881,bioterrorism,NaN,To fight bioterrorism sir.,1
620,898,bioterrorism,NaN,To fight bioterrorism sir.,0
1190,1723,buildings%20burning,"Mackay, QLD, Australia",Mmmmmm I'm burning.... I'm burning buildings I...,1
1207,1752,buildings%20burning,"Epic City, BB.",I Pledge Allegiance To The P.O.P.E. And The Bu...,0
1214,1760,buildings%20burning,dallas,like for the music video I want some real acti...,1
1323,1922,burning%20buildings,"Mackay, QLD, Australia",Mmmmmm I'm burning.... I'm burning buildings I...,0
1337,1950,burning%20buildings,dallas,like for the music video I want some real acti...,0
1351,1968,burning%20buildings,"Epic City, BB.",I Pledge Allegiance To The P.O.P.E. And The Bu...,1
2802,4068,displaced,Pedophile hunting ground,.POTUS #StrategicPatience is a strategy for #G...,1
2803,4076,displaced,Pedophile hunting ground,.POTUS #StrategicPatience is a strategy for #G...,0


In [5]:
df.drop_duplicates(subset = ['text'], keep = False, inplace = True, ignore_index = True)

In [6]:
duplicates_record = df[df.duplicated(['text'], keep=False)]
duplicates_record

,id,keyword,location,text,target


In [7]:
df.isna().sum()

id             0
keyword       56
location    2472
text           0
target         0
dtype: int64

In [8]:
df.shape

(7485, 5)

In [9]:
df['keyword'] = np.where(df['keyword'].isna(),'missing', df['keyword'])
df

,id,keyword,location,text,target
0,1,missing,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,missing,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,missing,NaN,All residents asked to 'shelter in place' are ...,1
3,6,missing,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,missing,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7480,10863,missing,NaN,#WorldNews Fallen powerlines on G:link tram: U...,1
7481,10864,missing,NaN,on the flip side I'm at Walmart and there is a...,1
7482,10866,missing,NaN,Suicide bomber kills 15 in Saudi security site...,1
7483,10869,missing,NaN,Two giant cranes holding a bridge collapse int...,1


In [10]:
df['target'].value_counts()

0    4297
1    3188
Name: target, dtype: int64

In [11]:
df=df[['keyword','text','target']]

In [12]:
df.head()

,keyword,text,target
0,missing,Our Deeds are the Reason of this #earthquake M...,1
1,missing,Forest fire near La Ronge Sask. Canada,1
2,missing,All residents asked to 'shelter in place' are ...,1
3,missing,"13,000 people receive #wildfires evacuation or...",1
4,missing,Just got sent this photo from Ruby #Alaska as ...,1


In [13]:
!python -m spacy download en_core_web_sm -q

[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


2022-01-01 17:24:11.291755: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2022-01-01 17:24:11.291805: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [14]:
# Import Spacy and french initialisation
import en_core_web_sm
nlp = en_core_web_sm.load()

In [15]:
# Import Stop words 
from spacy.lang.en.stop_words import STOP_WORDS

In [16]:
df["text_clean"] = df["text"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" "))
df["text_clean"] = df["text_clean"].apply(lambda x: x.replace(" +"," ").lower().strip())
df["text_clean"] = df["text_clean"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) & (token.text not in STOP_WORDS)]))

df["keyword_clean"] = df["keyword"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" "))
df["keyword_clean"] = df["keyword_clean"].apply(lambda x: x.replace(" +"," ").lower().strip())
df["keyword_clean"] = df["keyword_clean"].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if (token.lemma_ not in STOP_WORDS) & (token.text not in STOP_WORDS)]))

<ipython-input-16-4b0479e6ce56>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_clean"] = df["text"].apply(lambda x:''.join(ch for ch in x if ch.isalnum() or ch==" "))
<ipython-input-16-4b0479e6ce56>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_clean"] = df["text_clean"].apply(lambda x: x.replace(" +"," ").lower().strip())
<ipython-input-16-4b0479e6ce56>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

In [17]:
mask = df["text_clean"].isna()==False
df = df[mask]

In [18]:
from spellchecker import SpellChecker

def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',text)

def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)


spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

In [19]:
df['text_clean']=df['text_clean'].apply(remove_URL)
df['text_clean']=df['text_clean'].apply(remove_emoji)
# df['text_clean']=df['text_clean'].apply(correct_spellings)

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from nltk.corpus import stopwords
from nltk.util import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from collections import defaultdict
from collections import  Counter
plt.style.use('ggplot')
stop=set(stopwords.words('english'))
import re
from nltk.tokenize import word_tokenize
import gensim
import string
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.initializers import Constant
from sklearn.model_selection import train_test_split
# from keras.optimizers import Adam

In [21]:
from tqdm import tqdm

def create_corpus(df):
    corpus=[]
    for tweet in tqdm(df['text_clean']):
        words=[word.lower() for word in word_tokenize(tweet) if((word.isalpha()==1) & (word not in stop))]
        corpus.append(words)
    return corpus

In [22]:
corpus=create_corpus(df)

100%|██████████| 7485/7485 [00:00<00:00, 8123.95it/s]


In [ ]:
df

In [ ]:
# y=tf.reshape(df.target,(-1,1))

In [24]:
import numpy as np
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000) # instanciate the tokenizer
tokenizer.fit_on_texts(df["text_clean"])
df["text_encoded"] = tokenizer.texts_to_sequences(df.text_clean)
df["len_review"] = df["text_encoded"].apply(lambda x: len(x))

df["keyword_encoded"] = tokenizer.texts_to_sequences(df.keyword_clean)
df["len_keyword"] = df["keyword_encoded"].apply(lambda x: len(x))

df = df[df["len_review"]!=0]
df = df[df["len_keyword"]!=0]

In [25]:
text_pad = tf.keras.preprocessing.sequence.pad_sequences(df.text_encoded, padding="post", maxlen=10)
keyword_pad = tf.keras.preprocessing.sequence.pad_sequences(df.keyword_encoded, padding="post")

In [26]:
text_pad.shape

(6250, 10)

In [27]:
full_ds = tf.data.Dataset.from_tensor_slices((text_pad, df.target))

In [28]:
full_ds

<TensorSliceDataset shapes: ((10,), ()), types: (tf.int32, tf.int64)>

In [29]:
# Train Test Split
TAKE_SIZE = int(0.8*df.shape[0])

train_data = full_ds.take(TAKE_SIZE).shuffle(TAKE_SIZE)
train_data = train_data.batch(16)

test_data = full_ds.skip(TAKE_SIZE)
test_data = test_data.batch(16)

In [30]:
 # Regardons un batch 
for text, targ in train_data.take(1):
  print(text, targ)

tf.Tensor(
[[  2 349 603   0   0   0   0   0   0   0]
 [  4   3   0   0   0   0   0   0   0   0]
 [ 95   0   0   0   0   0   0   0   0   0]
 [199 340 764 369 446  82 431   0   0   0]
 [ 31  97 687   0   0   0   0   0   0   0]
 [982 603 855 303  73  28 396   1  63   0]
 [458 555 555 235 251   0   0   0   0   0]
 [ 91 432 244   0   0   0   0   0   0   0]
 [ 55 189 736 867 188 579 477   0   0   0]
 [120 388   0   0   0   0   0   0   0   0]
 [122 506   0   0   0   0   0   0   0   0]
 [395 660 764 312 150 152   0   0   0   0]
 [ 32 876 330   2 407   0   0   0   0   0]
 [ 63  10 780 254 243 468  13   0   0   0]
 [124   6  41 197  10  87 694   0   0   0]
 [535 120  93   0   0   0   0   0   0   0]], shape=(16, 10), dtype=int32) tf.Tensor([0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 1], shape=(16,), dtype=int64)


In [31]:
vocab_size = len(tokenizer.word_index)
vocab_size

20345

In [32]:
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, GRU, LSTM, SpatialDropout1D, Dropout
from tensorflow.keras.regularizers import L2

max_words = 1000
max_len = 10

model=tf.keras.Sequential()

model.add(tf.keras.layers.Input(shape=[max_len]))
model.add(tf.keras.layers.Embedding(max_words,128,input_length=max_len))    

model.add(tf.keras.layers.LSTM(200, return_sequences=True))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.LSTM(200,return_sequences=True))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.LSTM(200))
model.add(tf.keras.layers.Dropout(0.5))
          
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Dropout(0.5))

model.add(tf.keras.layers.Dense(1,activation='sigmoid')) #output layer

In [33]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 128)           128000    
_________________________________________________________________
lstm (LSTM)                  (None, 10, 200)           263200    
_________________________________________________________________
dropout (Dropout)            (None, 10, 200)           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 200)           320800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 200)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 200)               320800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0

In [34]:
optimizer= tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer,
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy()])

In [35]:
model.fit(
    train_data,
    validation_data=test_data,
    epochs=10)

Epoch 1/10
313/313 [==============================] - 17s 32ms/step - loss: 0.5253 - binary_accuracy: 0.7536 - val_loss: 0.4846 - val_binary_accuracy: 0.7656
Epoch 2/10
313/313 [==============================] - 10s 32ms/step - loss: 0.4266 - binary_accuracy: 0.8218 - val_loss: 0.5282 - val_binary_accuracy: 0.7592
Epoch 3/10
313/313 [==============================] - 10s 33ms/step - loss: 0.4038 - binary_accuracy: 0.8330 - val_loss: 0.5496 - val_binary_accuracy: 0.7520
Epoch 4/10
313/313 [==============================] - 11s 35ms/step - loss: 0.3830 - binary_accuracy: 0.8392 - val_loss: 0.5454 - val_binary_accuracy: 0.7664
Epoch 5/10
313/313 [==============================] - 10s 32ms/step - loss: 0.3607 - binary_accuracy: 0.8480 - val_loss: 0.5048 - val_binary_accuracy: 0.7624
Epoch 6/10
313/313 [==============================] - 10s 33ms/step - loss: 0.3355 - binary_accuracy: 0.8564 - val_loss: 0.7358 - val_binary_accuracy: 0.7280
Epoch 7/10
313/313 [==============================] 

In [36]:
pred=model.predict(text_pad)

In [37]:
df['pred']=abs(pred)

In [38]:
def categ (n):
    if n <0.5:
        val=0
    else:
        val=1
    return val

df['pred2']=df['pred'].apply(categ)

In [39]:
df[df['target']==df['pred2']]

,keyword,text,target,text_clean,keyword_clean,text_encoded,len_review,keyword_encoded,len_keyword,pred,pred2
0,missing,Our Deeds are the Reason of this #earthquake M...,1,deed reason earthquake allah forgive,miss,"[408, 176]",2,[121],1,0.983850,1
1,missing,Forest fire near La Ronge Sask. Canada,1,forest fire near la ronge sask canada,miss,"[116, 3, 162, 507, 999]",5,[121],1,0.999245,1
3,missing,"13,000 people receive #wildfires evacuation or...",1,13000 people receive wildfire evacuation order...,miss,"[8, 67, 201, 299, 36]",5,[121],1,0.999939,1
4,missing,Just got sent this photo from Ruby #Alaska as ...,1,got send photo ruby alaska smoke wildfire pour...,miss,"[262, 177, 122, 167, 67, 112]",6,[121],1,0.999728,1
5,missing,#RockyFire Update => California Hwy. 20 closed...,1,rockyfire update california hwy 20 close dir...,miss,"[168, 36, 441, 315, 822, 773, 275, 3, 67]",9,[121],1,0.999677,1
...,...,...,...,...,...,...,...,...,...,...,...
7479,missing,Officials say a quarantine is in place at an A...,1,official quarantine place alabama home possibl...,miss,"[205, 96, 330, 935, 29, 378, 853, 397]",8,[121],1,0.999711,1
7480,missing,#WorldNews Fallen powerlines on G:link tram: U...,1,worldnews fall powerline glink tram update fir...,miss,"[84, 168, 3, 519, 83, 489, 814]",7,[121],1,0.999949,1
7482,missing,Suicide bomber kills 15 in Saudi security site...,1,suicide bomber kill 15 saudi security site mos...,miss,"[42, 155, 10, 318, 260, 399, 411, 322, 37, 9, ...",11,[121],1,0.989972,1
7483,missing,Two giant cranes holding a bridge collapse int...,1,giant crane hold bridge collapse nearby home h...,miss,"[590, 920, 308, 249, 38, 481, 29]",7,[121],1,0.999297,1


In [40]:
df[df['target']!=df['pred2']]

,keyword,text,target,text_clean,keyword_clean,text_encoded,len_review,keyword_encoded,len_keyword,pred,pred2
2,missing,All residents asked to 'shelter in place' are ...,1,resident ask shelter place notify officer evac...,miss,"[440, 330, 287, 201, 330, 299, 375]",7,[121],1,0.414820,0
7,missing,I'm on top of the hill and I can see a fire in...,1,m hill fire wood,miss,"[4, 3]",2,[121],1,0.248654,0
31,ablaze,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,bbcmtd wholesale market ablaze httptcolhyxeohy6c,ablaze,"[264, 442]",2,[442],1,0.317644,0
82,accident,#TruckCrash Overturns On #FortWorth Interstate...,1,truckcrash overturn fortworth interstate httpt...,accident,[88],1,[60],1,0.318912,0
89,accident,???? it was an accident http://t.co/Oia5fxi4gM,0,accident httptcooia5fxi4gm,accident,[60],1,[60],1,0.504342,1
...,...,...,...,...,...,...,...,...,...,...,...
7463,missing,#Sismo DETECTADO #JapÌ_n 15:41:07 Seismic inte...,1,sismo detectado japìn 154107 seismic intensity...,miss,[483],1,[121],1,0.274744,0
7469,missing,An IS group suicide bomber detonated an explos...,1,group suicide bomber detonate explosivespacked...,miss,"[323, 42, 155, 160, 322, 425, 260, 430, 10, 31...",11,[121],1,0.314901,0
7475,missing,Father-of-three Lost Control of Car After Over...,1,fatherofthree lose control car overtake collid...,miss,"[263, 715, 43, 93]",4,[121],1,0.010104,0
7478,missing,a siren just went off and it wasn't the Forney...,1,siren nt forney tornado warn,miss,"[179, 1, 331, 457]",4,[121],1,0.083323,0
